# **RDDM Repo Demonstration**
### *https://github.com/anantaraha/RDDM (Forked from: [DebadityaQU](https://github.com/DebadityaQU/RDDM))*
- **Download necessary datasets**
- **Preprocess data**
- **Train DDPM (T=10, 25, 50) and RDDM (T=10) shortly, for demo purpose**
- **Standard evaluation (Complete)**
- **Cardiobench (Heart Rate and Atrial Fibrillation detection)**

In [1]:
# Clean incompatible versions
!pip uninstall -y numpy similaritymeasures

# Install compatible versions
!pip install -U pip setuptools wheel
!pip install numpy==1.26.4 scipy==1.10.1
!pip install neurokit2==0.2.7 biosppy==1.0.0 torchmetrics==1.3.2 wfdb==4.1.2
!pip install scikit-learn matplotlib pandas einops pyyaml gdown tqdm
!pip install similaritymeasures==0.4.4
!pip install mat73 h5py

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.21.0 requires numpy>=1.23.0, which is not installed.
fasttext 0.9.3 requires numpy, which is not installed.
pymc3 3.11.4 requires numpy>=1.15.0, which is not installed.
pyldavis 3.4.1 requires numpy>=1.24.2, which is not installed.
open-spi

In [2]:
# Download repository
%rm -r RDDM
!git clone 'https://github.com/anantaraha/RDDM.git'

%mkdir weights
%mkdir datasets

%cd RDDM
%ls

rm: cannot remove 'RDDM': No such file or directory
Cloning into 'RDDM'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 133 (delta 78), reused 80 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (133/133), 36.47 MiB | 48.50 MiB/s, done.
Resolving deltas: 100% (78/78), done.
/kaggle/working/RDDM
cardiobench.py  diffusion.py  lr_scheduler.py  preprocessing.py  std_eval.sh
config.py       eval-logs/    metrics.py       README.md         train.py
data.py         LICENSE       model.py         std_eval.py


# Fetch Datasets

In [3]:
# Download dataset: WESAD
!wget -O 'wesad.zip' 'https://uni-siegen.sciebo.de/s/HGdUkoNlW1Ub0Gx/download/WESAD.zip'

--2025-11-19 15:52:41--  https://uni-siegen.sciebo.de/s/HGdUkoNlW1Ub0Gx/download/WESAD.zip
Resolving uni-siegen.sciebo.de (uni-siegen.sciebo.de)... 128.176.1.4
Connecting to uni-siegen.sciebo.de (uni-siegen.sciebo.de)|128.176.1.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2249444501 (2.1G) [application/zip]
Saving to: ‘wesad.zip’

wesad.zip           100%[===================>]   2.09G  36.2MB/s    in 60s     

2025-11-19 15:53:42 (35.7 MB/s) - ‘wesad.zip’ saved [2249444501/2249444501]



In [4]:
# Download dataset: CAPNO
!gdown 'https://drive.google.com/uc?id=1HWTAWT1phMQYZlnKJ160FnEPAaRECbk2' -O 'capno.zip'

Downloading...
From (original): https://drive.google.com/uc?id=1HWTAWT1phMQYZlnKJ160FnEPAaRECbk2
From (redirected): https://drive.google.com/uc?id=1HWTAWT1phMQYZlnKJ160FnEPAaRECbk2&confirm=t&uuid=dc877e58-3cfe-48eb-bb1a-54e77e423ca7
To: /kaggle/working/RDDM/capno.zip
100%|██████████████████████████████████████| 57.8M/57.8M [00:01<00:00, 35.5MB/s]


In [5]:
# Download dataset: DALIA
!wget -O 'dalia.zip' 'https://uni-siegen.sciebo.de/s/pfHzlTepXkiJ4jP/download/PPG_FieldStudy.zip'

--2025-11-19 15:53:46--  https://uni-siegen.sciebo.de/s/pfHzlTepXkiJ4jP/download/PPG_FieldStudy.zip
Resolving uni-siegen.sciebo.de (uni-siegen.sciebo.de)... 128.176.1.4
Connecting to uni-siegen.sciebo.de (uni-siegen.sciebo.de)|128.176.1.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2864337783 (2.7G) [application/zip]
Saving to: ‘dalia.zip’

dalia.zip           100%[===================>]   2.67G  35.4MB/s    in 79s     

2025-11-19 15:55:05 (34.6 MB/s) - ‘dalia.zip’ saved [2864337783/2864337783]



In [6]:
# Download dataset: BIDMC
!wget -O bidmc.zip 'https://physionet.org/content/bidmc/get-zip/1.0.0/'

--2025-11-19 15:55:06--  https://physionet.org/content/bidmc/get-zip/1.0.0/
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217902224 (208M) [application/zip]
Saving to: ‘bidmc.zip’

bidmc.zip           100%[===================>] 207.81M   353KB/s    in 10m 0s  

2025-11-19 16:05:06 (355 KB/s) - ‘bidmc.zip’ saved [217902224/217902224]



In [7]:
# Download dataset: MIMIC-AFib
!wget -O 'mimic1.mat' 'https://zenodo.org/record/6807403/files/mimic_perform_af_data.mat?download=1'
!wget -O 'mimic2.mat' 'https://zenodo.org/record/6807403/files/mimic_perform_non_af_data.mat?download=1'

--2025-11-19 16:05:06--  https://zenodo.org/record/6807403/files/mimic_perform_af_data.mat?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.75, 188.185.43.153, 137.138.52.235, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.75|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/6807403/files/mimic_perform_af_data.mat [following]
--2025-11-19 16:05:07--  https://zenodo.org/records/6807403/files/mimic_perform_af_data.mat
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 20603819 (20M) [application/octet-stream]
Saving to: ‘mimic1.mat’

mimic1.mat          100%[===================>]  19.65M  1.20MB/s    in 20s     

2025-11-19 16:05:27 (1002 KB/s) - ‘mimic1.mat’ saved [20603819/20603819]

--2025-11-19 16:05:27--  https://zenodo.org/record/6807403/files/mimic_perform_non_af_data.mat?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.153, 137.138.52.235, 188.185.48.75, 

# Apply Preprocessing
- Downloaded datasets must be processed into defined window and sampling rates for evaluation
- Frequency=128 Hz, Window=4 seconds

In [8]:
!python preprocessing.py

WESAD (Expects wesad.zip in current directory): https://uni-siegen.sciebo.de/s/HGdUkoNlW1Ub0Gx/download/WESAD.zip
--------------------------------------------------------------------------------
WESAD/S10/S10.pkl
WESAD/S11/S11.pkl
WESAD/S13/S13.pkl
WESAD/S14/S14.pkl
WESAD/S15/S15.pkl
WESAD/S16/S16.pkl
WESAD/S17/S17.pkl
WESAD/S2/S2.pkl
WESAD/S3/S3.pkl
WESAD/S4/S4.pkl
WESAD/S5/S5.pkl
WESAD/S6/S6.pkl
WESAD/S7/S7.pkl
WESAD/S8/S8.pkl
WESAD/S9/S9.pkl
Saved: ['../datasets/WESAD/ppg_test_4sec.npy', '../datasets/WESAD/ppg_train_4sec.npy', '../datasets/WESAD/ecg_test_4sec.npy', '../datasets/WESAD/ecg_train_4sec.npy']
Saved: ['../datasets/WESAD/ecg_test_8sec.npy', '../datasets/WESAD/ecg_train_8sec.npy', '../datasets/WESAD/ppg_train_8sec.npy', '../datasets/WESAD/ppg_test_8sec.npy']
CAPNO (Expects capno.zip in current directory): https://drive.google.com/uc?id=1HWTAWT1phMQYZlnKJ160FnEPAaRECbk2
--------------------------------------------------------------------------------
data/mat/0103_8min.mat
da

# Train DDPM
- DDPM short training for demonstration
- Save weights for evaluation (next)
- nT = 10, 25, 50

In [9]:
# DDPM training with nT=10
!python train.py ddpm 10

# Rename generated weights accordingly
!mv '../weights/DDPM_epoch4.pth' '../weights/ddpm_main_network_10.pth'
!mv '../weights/ConditionNet_epoch4.pth' '../weights/ddpm_condition_encoder_10.pth'

Deterministic with seed = 31

****************** Epoch - 0 *******************


loss: 0.1101: 100%|████████████████| 950/950 [05:34<00:00,  2.84it/s, DDPM=0.11]

****************** Epoch - 1 *******************


loss: 0.0651: 100%|██████████████| 950/950 [05:32<00:00,  2.86it/s, DDPM=0.0651]

****************** Epoch - 2 *******************


loss: 0.0398: 100%|██████████████| 950/950 [05:33<00:00,  2.85it/s, DDPM=0.0398]

****************** Epoch - 3 *******************


loss: 0.0496: 100%|██████████████| 950/950 [05:34<00:00,  2.84it/s, DDPM=0.0496]

****************** Epoch - 4 *******************


loss: 0.0424: 100%|██████████████| 950/950 [05:33<00:00,  2.85it/s, DDPM=0.0424]


In [10]:
# DDPM training with nT=25
!python train.py ddpm 25

# Rename generated weights accordingly
!mv '../weights/DDPM_epoch4.pth' '../weights/ddpm_main_network_25.pth'
!mv '../weights/ConditionNet_epoch4.pth' '../weights/ddpm_condition_encoder_25.pth'

Deterministic with seed = 31

****************** Epoch - 0 *******************


loss: 0.0852: 100%|██████████████| 950/950 [05:33<00:00,  2.85it/s, DDPM=0.0852]

****************** Epoch - 1 *******************


loss: 0.0525: 100%|██████████████| 950/950 [05:33<00:00,  2.85it/s, DDPM=0.0525]

****************** Epoch - 2 *******************


loss: 0.0413: 100%|██████████████| 950/950 [05:32<00:00,  2.85it/s, DDPM=0.0413]

****************** Epoch - 3 *******************


loss: 0.0361: 100%|██████████████| 950/950 [05:32<00:00,  2.86it/s, DDPM=0.0361]

****************** Epoch - 4 *******************


loss: 0.0342: 100%|██████████████| 950/950 [05:33<00:00,  2.85it/s, DDPM=0.0342]


In [11]:
# DDPM training with nT=50
!python train.py ddpm 50

# Rename generated weights accordingly
!mv '../weights/DDPM_epoch4.pth' '../weights/ddpm_main_network_50.pth'
!mv '../weights/ConditionNet_epoch4.pth' '../weights/ddpm_condition_encoder_50.pth'

Deterministic with seed = 31

****************** Epoch - 0 *******************


loss: 0.0887: 100%|██████████████| 950/950 [05:33<00:00,  2.85it/s, DDPM=0.0887]

****************** Epoch - 1 *******************


loss: 0.0354: 100%|██████████████| 950/950 [05:33<00:00,  2.85it/s, DDPM=0.0354]

****************** Epoch - 2 *******************


loss: 0.0304: 100%|██████████████| 950/950 [05:32<00:00,  2.85it/s, DDPM=0.0304]

****************** Epoch - 3 *******************


loss: 0.0246: 100%|██████████████| 950/950 [05:32<00:00,  2.86it/s, DDPM=0.0246]

****************** Epoch - 4 *******************


loss: 0.0261: 100%|██████████████| 950/950 [05:32<00:00,  2.85it/s, DDPM=0.0261]


# Train RDDM
- RDDM short training
- Saved weights will be ignored, since the downloaded weights are used for evaluation later.
- nT = 10 (Fixed)

In [12]:
# RDDM training (fixed nT=10 as the paper)
!python train.py rddm

Deterministic with seed = 31

****************** Epoch - 0 *******************


loss: 5.4581: 100%|██| 950/950 [10:58<00:00,  1.44it/s, DDPM=5.4, Region=0.0616]

****************** Epoch - 1 *******************


loss: 2.9731: 100%|█| 950/950 [10:56<00:00,  1.45it/s, DDPM=2.93, Region=0.0455]

****************** Epoch - 2 *******************


loss: 2.3807: 100%|█| 950/950 [10:57<00:00,  1.44it/s, DDPM=2.33, Region=0.0531]

****************** Epoch - 3 *******************


loss: 1.7322: 100%|█| 950/950 [10:57<00:00,  1.44it/s, DDPM=1.68, Region=0.0488]

****************** Epoch - 4 *******************


loss: 1.5678: 100%|█| 950/950 [10:57<00:00,  1.44it/s, DDPM=1.52, Region=0.0491]


# Standard Evaluation: Baseline DDPM
- Naive DDPM with nT=10, 25, 50
- Using our saved weights:
  - *ddpm_main_network_10.pth & ddpm_condition_encoder_10.pth*
  - *ddpm_main_network_25.pth & ddpm_condition_encoder_25.pth*
  - *ddpm_main_network_50.pth & ddpm_condition_encoder_50.pth*

In [13]:
# Naive DDPM, nT=10
!python std_eval.py ddpm 10

Deterministic with seed = 31

******* Standard evaluation (Table 2, DDPM) results *******
100%|█████████████████████████████████████████████| 9/9 [01:06<00:00,  7.40s/it]

WESAD: RMSE is 0.3487287600304346, FD is 19.14774267551156
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
# Naive DDPM, nT=25
!python std_eval.py ddpm 25

Deterministic with seed = 31

******* Standard evaluation (Table 2, DDPM) results *******
100%|█████████████████████████████████████████████| 9/9 [02:20<00:00, 15.57s/it]

WESAD: RMSE is 0.2865998806742266, FD is 8.451546170914819
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
# Naive DDPM, nT=50
!python std_eval.py ddpm 50

Deterministic with seed = 31

******* Standard evaluation (Table 2, DDPM) results *******
100%|█████████████████████████████████████████████| 9/9 [04:22<00:00, 29.16s/it]

WESAD: RMSE is 0.29095805995083185, FD is 8.491031474735127
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Standard Evaluation: RDDM
- RDDM with fixed nT=10 as paper
- Download pretrained weights:
  - *rddm_condition_encoder_1.pth, rddm_condition_encoder_2.pth, rddm_main_network.pth*

In [16]:
# Weights download from drive
!gdown 'https://drive.google.com/uc?id=1RWe2Dm08WceX_H3x0ECaoP1IT2DdoQcn' -O ../weights/rddm_condition_encoder_1.pth
!gdown 'https://drive.google.com/uc?id=19xICld-We-EJnct2ZD6M-YpWwq8Tl3cg' -O ../weights/rddm_condition_encoder_2.pth
!gdown 'https://drive.google.com/uc?id=1PxLMTECuupTiGCmA3zhdxtqQJ7IWL88B' -O ../weights/rddm_main_network.pth


Downloading...
From (original): https://drive.google.com/uc?id=1RWe2Dm08WceX_H3x0ECaoP1IT2DdoQcn
From (redirected): https://drive.google.com/uc?id=1RWe2Dm08WceX_H3x0ECaoP1IT2DdoQcn&confirm=t&uuid=9f46b7e3-7a5b-4d4a-9b40-d5f13f3e0e05
To: /kaggle/working/weights/rddm_condition_encoder_1.pth
100%|█████████████████████████████████████████| 108M/108M [00:01<00:00, 101MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=19xICld-We-EJnct2ZD6M-YpWwq8Tl3cg
From (redirected): https://drive.google.com/uc?id=19xICld-We-EJnct2ZD6M-YpWwq8Tl3cg&confirm=t&uuid=d31f44f8-a5e1-4fa8-85df-da1311e1b4d6
To: /kaggle/working/weights/rddm_condition_encoder_2.pth
100%|████████████████████████████████████████| 108M/108M [00:01<00:00, 89.7MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1PxLMTECuupTiGCmA3zhdxtqQJ7IWL88B
From (redirected): https://drive.google.com/uc?id=1PxLMTECuupTiGCmA3zhdxtqQJ7IWL88B&confirm=t&uuid=5ec40cbd-0c9d-474b-9ab0-0f6a1073ded2
To: /kaggle/working/weigh

In [17]:
# RDDM, nt10
!python std_eval.py rddm

Deterministic with seed = 31

******* Standard evaluation (Table 2, RDDM) results *******
100%|█████████████████████████████████████████████| 9/9 [01:57<00:00, 13.03s/it]

WESAD: RMSE is 0.2213003948187234, FD is 3.6112384716840387
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Cardiobench: DDPM
- Currently implemented: Heart Rate, Atrial Fibrillation Detection
- Naive DDPM with nT=50
- Using our saved weights: 
  - *ddpm_main_network_50.pth & ddpm_condition_encoder_50.pth*

In [18]:
# Naive DDPM, with nT=10
!python cardiobench.py ddpm 50

Deterministic with seed = 31
Deterministic with seed = 31

******* Heart Rate estimation (Table 3, DDPM) results *******
100%|█████████████████████████████████████████████| 5/5 [04:16<00:00, 51.24s/it]

WESAD: Mean Absolute Error (BPM) is 15.73768704722691
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Cardiobench: RDDM
- RDDM with fixed nT=10 as paper
- Download pretrained weights: ***rddm_condition_encoder_1.pth, rddm_condition_encoder_2.pth, rddm_main_network.pth***

In [19]:
# RDDM, fixed nT=10
!python cardiobench.py rddm

Deterministic with seed = 31
Deterministic with seed = 31

******* Heart Rate estimation (Table 3, RDDM) results *******
100%|█████████████████████████████████████████████| 5/5 [01:50<00:00, 22.17s/it]

WESAD: Mean Absolute Error (BPM) is 1.9647095403523573
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------